In [ ]:
from google.colab import drive
import pandas as pd
from IPython.display import display
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import DataLoader, TensorDataset

drive.mount('/content/drive')

old_churches_1 = '/content/drive/MyDrive/OregonChurches/Old_Churches_Oregon_1.csv'
old_churches_2 = '/content/drive/MyDrive/OregonChurches/Old_Churches_Oregon_2.csv'
old_churches_3 = '/content/drive/MyDrive/OregonChurches/Old_Churches_Oregon_3.csv'
latest_churches = '/content/drive/MyDrive/OregonChurches/Churches_Latest.csv'

df_old_1 = pd.read_csv(old_churches_1)
df_old_2 = pd.read_csv(old_churches_2)
df_old_3 = pd.read_csv(old_churches_3)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_old_1.columns = df_old_1.columns.str.strip()
df_old_2.columns = df_old_2.columns.str.strip()
df_old_3.columns = df_old_3.columns.str.strip()

common_columns = ['Business Name', 'Entity Type', 'Nonprofit Type', 'Registry Date',
                  'Associated Name Type', 'Address', 'City', 'State', 'Zip Code']

common_columns_1 = [col for col in common_columns if col in df_old_1.columns]
common_columns_2 = [col for col in common_columns if col in df_old_2.columns]
common_columns_3 = [col for col in common_columns if col in df_old_3.columns]

df_old_1_filtered = df_old_1[common_columns_1]
df_old_2_filtered = df_old_2[common_columns_2]
df_old_3_filtered = df_old_3[common_columns_3]
df_old_merged = pd.concat([df_old_1_filtered, df_old_2_filtered, df_old_3_filtered], ignore_index=True)

df_latest = pd.read_csv(latest_churches)
df_latest.columns = df_latest.columns.str.strip()
df_latest_renamed = df_latest.rename(columns={
    'NAME': 'Business Name',
    'STREET': 'Address',
    'CITY': 'City',
    'STATE': 'State',
    'ZIP': 'Zip Code',
})

df_latest_filtered = df_latest_renamed[[col for col in common_columns if col in df_latest_renamed.columns]]
df_latest_filtered = df_latest_filtered[df_latest_filtered['Business Name'].str.contains('church', case=False, na=False)]
df_latest_filtered_file = '/content/drive/MyDrive/OregonChurches/Churches_Latest.csv'
df_latest_filtered.to_csv(df_latest_filtered_file, index=False)

display(df_old_merged.head())
display(df_latest_filtered.head())

,Business Name,Entity Type,Nonprofit Type,Registry Date,Associated Name Type,Address,City,State,Zip Code
0,THURSTON CHRISTIAN CHURCH,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1892-12-01 00:00:00,PRINCIPAL PLACE OF BUSINESS,6680 THURSTON RD,SPRINGFIELD,OR,97478
1,CENTENNIAL CHRISTIAN CENTER,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1897-04-23 00:00:00,PRINCIPAL PLACE OF BUSINESS,1020 JANUS ST,SPRINGFIELD,OR,97477
2,"FIRST BAPTIST CHURCH OF SPRINGFIELD, OREGON",DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1914-11-07 00:00:00,PRINCIPAL PLACE OF BUSINESS,1175 G ST,SPRINGFIELD,OR,97477
3,NEW LIFE CHURCH FIRST ASSEMBLY OF GOD SPRINGFIELD,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1940-12-30 00:00:00,PRINCIPAL PLACE OF BUSINESS,2080 N 19TH ST,SPRINGFIELD,OR,97477
4,FIRST CHRISTIAN CHURCH OF SPRINGFIELD,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1942-09-24 00:00:00,PRINCIPAL PLACE OF BUSINESS,395 W CENTENNIAL BLVD,SPRINGFIELD,OR,97477


,Business Name,Address,City,State,Zip Code
0,HILLSBORO VINEYARD CHURCH,511 SW 211TH AVE,BEAVERTON,OR,97006-6420
1,SKYLINE BAPTIST CHURCH,4787 SKYLINE RD S,SALEM,OR,97306-2404
2,VERTICAL CHURCH,PO BOX 707,LA PINE,OR,97739-0707
3,CREEKSIDE COMMUNITY CHURCH,PO BOX 1710,SILVERTON,OR,97381-0365
4,MIRROR POND FREE METHODIST CHURCH,PO BOX 214,BEND,OR,97709-0214


In [18]:
remaining_churches = df_old_merged[df_old_merged['Business Name'].isin(df_latest_filtered['Business Name'])]
remaining_churches_cleaned = remaining_churches.drop_duplicates(subset=['Business Name'])
remaining_churches_file = '/content/drive/MyDrive/OregonChurches/Remaining_Churches_Cleaned.csv'
remaining_churches_cleaned.to_csv(remaining_churches_file, index=False)
# display(remaining_churches_cleaned.head())

churches_no_longer_exist = df_old_merged[~df_old_merged['Business Name'].isin(df_latest_filtered['Business Name'])]
churches_no_longer_exist_cleaned = churches_no_longer_exist.drop_duplicates(subset=['Business Name'])
no_longer_exist_file = '/content/drive/MyDrive/OregonChurches/Churches_No_Longer_Exist_Cleaned.csv'
churches_no_longer_exist_cleaned.to_csv(no_longer_exist_file, index=False)

# Turns out, money has nothing to do with the survival of the church.
# oregon_businesses_file = '/content/drive/MyDrive/OregonChurches/eo_or.csv'
# oregon_business_dataframe = pd.read_csv(oregon_businesses_file)
# oregon_business_dataframe['NAME'] = oregon_business_dataframe['NAME'].str.lower()
# churches_no_longer_exist_cleaned['Business Name'] = churches_no_longer_exist_cleaned['Business Name'].str.lower()
# remaining_churches_money = pd.merge(churches_no_longer_exist_cleaned, oregon_business_dataframe, left_on='Business Name', right_on='NAME', how='inner')
# asset_sorted = remaining_churches_money[['Business Name', 'ASSET_AMT', 'INCOME_AMT', 'REVENUE_AMT']].sort_values(by='ASSET_AMT', ascending=False)
# display(asset_sorted)
# income_sorted = remaining_churches_money[['Business Name', 'ASSET_AMT', 'INCOME_AMT', 'REVENUE_AMT']].sort_values(by='INCOME_AMT', ascending=False)
# display(income_sorted)
# revenue_sorted = remaining_churches_money[['Business Name', 'ASSET_AMT', 'INCOME_AMT', 'REVENUE_AMT']].sort_values(by='REVENUE_AMT', ascending=False)
# display(revenue_sorted)
# remaining_churches_cleaned['Business Name'] = remaining_churches_cleaned['Business Name'].str.lower()
# remaining_churches_money = pd.merge(remaining_churches_cleaned, oregon_business_dataframe, left_on='Business Name', right_on='NAME', how='inner')
# asset_sorted = remaining_churches_money[['Business Name', 'ASSET_AMT', 'INCOME_AMT', 'REVENUE_AMT']].sort_values(by='ASSET_AMT', ascending=False)
# display(asset_sorted)
# income_sorted = remaining_churches_money[['Business Name', 'ASSET_AMT', 'INCOME_AMT', 'REVENUE_AMT']].sort_values(by='INCOME_AMT', ascending=False)
# display(income_sorted)
# revenue_sorted = remaining_churches_money[['Business Name', 'ASSET_AMT', 'INCOME_AMT', 'REVENUE_AMT']].sort_values(by='REVENUE_AMT', ascending=False)
# display(revenue_sorted)

<ipython-input-18-6b7eaed1b7bb>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  churches_no_longer_exist_cleaned['Business Name'] = churches_no_longer_exist_cleaned['Business Name'].str.lower()


,Business Name,ASSET_AMT,INCOME_AMT,REVENUE_AMT
19,village missions,5481807.0,8965574.0,8833787.0
20,west hills christian school,5305326.0,4228383.0,4134700.0
32,emissaries of divine light northwest,1512625.0,287932.0,32531.0
69,jehovah jireh international mission,1314007.0,601562.0,601562.0
118,russian old believers,956592.0,78944.0,78944.0
...,...,...,...,...
99,peace fellowship,NaN,NaN,NaN
104,victorious life fellowship,NaN,NaN,NaN
105,agape community ministries,NaN,NaN,NaN
106,international evangelistic ministry,NaN,NaN,NaN


,Business Name,ASSET_AMT,INCOME_AMT,REVENUE_AMT
19,village missions,5481807.0,8965574.0,8833787.0
95,mission increase,296142.0,7000480.0,7000480.0
20,west hills christian school,5305326.0,4228383.0,4134700.0
108,youth with a mission-slavic ministries interna...,455832.0,1595461.0,1595461.0
2,sunrise international ministries,428086.0,699455.0,699455.0
...,...,...,...,...
99,peace fellowship,NaN,NaN,NaN
104,victorious life fellowship,NaN,NaN,NaN
105,agape community ministries,NaN,NaN,NaN
106,international evangelistic ministry,NaN,NaN,NaN


,Business Name,ASSET_AMT,INCOME_AMT,REVENUE_AMT
19,village missions,5481807.0,8965574.0,8833787.0
95,mission increase,296142.0,7000480.0,7000480.0
20,west hills christian school,5305326.0,4228383.0,4134700.0
108,youth with a mission-slavic ministries interna...,455832.0,1595461.0,1595461.0
2,sunrise international ministries,428086.0,699455.0,699455.0
...,...,...,...,...
99,peace fellowship,NaN,NaN,NaN
104,victorious life fellowship,NaN,NaN,NaN
105,agape community ministries,NaN,NaN,NaN
106,international evangelistic ministry,NaN,NaN,NaN


<ipython-input-18-6b7eaed1b7bb>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_churches_cleaned['Business Name'] = remaining_churches_cleaned['Business Name'].str.lower()


,Business Name,ASSET_AMT,INCOME_AMT,REVENUE_AMT
61,korean mission church of oregon,2665172.0,362264.0,362264.0
59,korean first southern baptist church,2453070.0,190766.0,190766.0
83,oromo evangelical church of portland,561671.0,350820.0,350820.0
51,interchurch center,240659.0,81474.0,81474.0
82,oregon myanmar christian church,227681.0,57168.0,57168.0
...,...,...,...,...
116,all city church,NaN,NaN,NaN
117,vietnamese hope baptist church,NaN,NaN,NaN
118,bloom church,NaN,NaN,NaN
119,slavic christian church of salem,NaN,NaN,NaN


,Business Name,ASSET_AMT,INCOME_AMT,REVENUE_AMT
61,korean mission church of oregon,2665172.0,362264.0,362264.0
83,oromo evangelical church of portland,561671.0,350820.0,350820.0
59,korean first southern baptist church,2453070.0,190766.0,190766.0
51,interchurch center,240659.0,81474.0,81474.0
82,oregon myanmar christian church,227681.0,57168.0,57168.0
...,...,...,...,...
116,all city church,NaN,NaN,NaN
117,vietnamese hope baptist church,NaN,NaN,NaN
118,bloom church,NaN,NaN,NaN
119,slavic christian church of salem,NaN,NaN,NaN


,Business Name,ASSET_AMT,INCOME_AMT,REVENUE_AMT
61,korean mission church of oregon,2665172.0,362264.0,362264.0
83,oromo evangelical church of portland,561671.0,350820.0,350820.0
59,korean first southern baptist church,2453070.0,190766.0,190766.0
51,interchurch center,240659.0,81474.0,81474.0
82,oregon myanmar christian church,227681.0,57168.0,57168.0
...,...,...,...,...
116,all city church,NaN,NaN,NaN
117,vietnamese hope baptist church,NaN,NaN,NaN
118,bloom church,NaN,NaN,NaN
119,slavic christian church of salem,NaN,NaN,NaN


In [ ]:
remaining_churches = pd.read_csv('/content/drive/MyDrive/OregonChurches/Remaining_Churches_Cleaned.csv')
churches_no_longer_exist = pd.read_csv('/content/drive/MyDrive/OregonChurches/Churches_No_Longer_Exist_Cleaned.csv')

remaining_churches['Longevity'] = 1
churches_no_longer_exist['Longevity'] = 0

df = pd.concat([remaining_churches, churches_no_longer_exist], ignore_index=True)

label_encoder = LabelEncoder()
df['City'] = label_encoder.fit_transform(df['City'])
df['State'] = label_encoder.fit_transform(df['State'])

X = df[['City', 'State', 'Zip Code']]
y = df['Longevity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

class ChurchLongevityNN(nn.Module):
    def __init__(self):
        super(ChurchLongevityNN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

model = ChurchLongevityNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy: {accuracy * 100:.2f}%')

Epoch [1/50], Loss: 0.5835
Epoch [2/50], Loss: 0.4621
Epoch [3/50], Loss: 0.3796
Epoch [4/50], Loss: 0.3629
Epoch [5/50], Loss: 0.3606
Epoch [6/50], Loss: 0.3609
Epoch [7/50], Loss: 0.3559
Epoch [8/50], Loss: 0.3712
Epoch [9/50], Loss: 0.3608
Epoch [10/50], Loss: 0.3609
Epoch [11/50], Loss: 0.3639
Epoch [12/50], Loss: 0.3603
Epoch [13/50], Loss: 0.3604
Epoch [14/50], Loss: 0.3653
Epoch [15/50], Loss: 0.3596
Epoch [16/50], Loss: 0.3592
Epoch [17/50], Loss: 0.3653
Epoch [18/50], Loss: 0.3561
Epoch [19/50], Loss: 0.3607
Epoch [20/50], Loss: 0.3605
Epoch [21/50], Loss: 0.3698
Epoch [22/50], Loss: 0.3594
Epoch [23/50], Loss: 0.3650
Epoch [24/50], Loss: 0.3649
Epoch [25/50], Loss: 0.3552
Epoch [26/50], Loss: 0.3638
Epoch [27/50], Loss: 0.3562
Epoch [28/50], Loss: 0.3646
Epoch [29/50], Loss: 0.3606
Epoch [30/50], Loss: 0.3557
Epoch [31/50], Loss: 0.3557
Epoch [32/50], Loss: 0.3610
Epoch [33/50], Loss: 0.3683
Epoch [34/50], Loss: 0.3607
Epoch [35/50], Loss: 0.3682
Epoch [36/50], Loss: 0.3693
E